In [2]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor as XGBR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import log_loss
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import shap
import seaborn as sns
import numpy as np
import pandas as pd
import os

/home/chl/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
X = pd.read_excel('nanoplastic_std_x0.xlsx')
y = pd.read_excel('nanoplastic_std_y.xlsx')['Cell viability']
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=257)



In [20]:
for i in range(500):
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=i)
    rfc = DecisionTreeRegressor()
    CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    regressor = rfc.fit(Xtrain, Ytrain)
    score_test = regressor.score(Xtest,Ytest)
    print("5cv:",CV_score,"TEST:",score_test)
    if CV_score>0.6 and score_test>0.65:
        print("5cv:",CV_score,"TEST:",score_test,"random_state:",i)

5cv: 0.5083146202112475 TEST: 0.654747251961034
5cv: 0.6182146894036634 TEST: 0.5920421492403456
5cv: 0.5615484263967381 TEST: 0.600678027671219
5cv: 0.626258030529511 TEST: 0.5462966693507221
5cv: 0.47375745066236263 TEST: 0.6930914246907987
5cv: 0.5295516324054246 TEST: 0.6305478272148308
5cv: 0.4614061861348501 TEST: 0.5684029351302289
5cv: 0.557182412037518 TEST: 0.7164462415162289
5cv: 0.5062299965027208 TEST: 0.6452363034880839
5cv: 0.5669026413610514 TEST: 0.6577569587302545
5cv: 0.599232694756932 TEST: 0.7008347603062504
5cv: 0.519088295729975 TEST: 0.6905229447240048
5cv: 0.6072665639267248 TEST: 0.48758578270324904
5cv: 0.5499739841380136 TEST: 0.6238590444771475
5cv: 0.5770170007453127 TEST: 0.6890997053424619
5cv: 0.520898706877586 TEST: 0.5836851103246985
5cv: 0.5333306547869138 TEST: 0.6860572280400191
5cv: 0.5406848381130025 TEST: 0.6586455645722875
5cv: 0.44444406112146806 TEST: 0.6436474285934624
5cv: 0.5555555513992351 TEST: 0.6802657299593815
5cv: 0.5350576827920444 

In [21]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,y,test_size=0.2,random_state=257)

rfc = DecisionTreeRegressor()
CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
regressor = rfc.fit(Xtrain, Ytrain)
CV_predictions = cross_val_predict(rfc, Xtrain,Ytrain,cv=5)
rmse = np.sqrt(mean_squared_error(Ytrain,CV_predictions))


score_test = regressor.score(Xtest,Ytest)
print("5cv:",CV_score,"TEST:",score_test)
print("rmse_5CV",rmse)

5cv: 0.58324498564613 TEST: 0.7137686369755705
rmse_5CV 15.452865666337129


In [22]:
score_5cv_all = []
for i in range(0, 200, 1):
    rfc =DecisionTreeRegressor(random_state=i)
    score_5cv =cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(random_state_5cv))


score_5cv_all = []
for i in range(1, 300, 1):
    rfc = DecisionTreeRegressor(random_state=random_state_5cv
                                ,max_depth=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
max_depth_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(max_depth_5cv))


score_5cv_all = []
for i in range(1,X.shape[1]+1):
    rfc = DecisionTreeRegressor(random_state=random_state_5cv
                                ,max_depth=max_depth_5cv
                                ,max_features=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass

score_max_5cv = max(score_5cv_all)
max_features_5cv = range(1, X.shape[1]+1)[score_5cv_all.index(score_max_5cv)]

print("Best_5cv score：{}".format(score_max_5cv),
      "max_features_5cv:{}".format(max_features_5cv))

Best_5cv score：0.6198137781721957 random_5cv:136
Best_5cv score：0.6216322110769961 max_depth_5cv:42
Best_5cv score：0.6216322110769961 max_features_5cv:173


In [4]:

rfc = DecisionTreeRegressor(
                                random_state=136
                                ,max_depth=42
                                ,max_features=173)

In [5]:
CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(rfc, Xtrain,Ytrain,cv=5)
rmse = np.sqrt(mean_squared_error(Ytrain,CV_predictions))
mae = MAE(Ytrain,CV_predictions)
expvspred_5cv = {'Exp': Ytrain, 'Pred':CV_predictions}
print("r2_5cv:",CV_score,"rmse_5CV",rmse,"mae_5CV",mae)
pd.DataFrame(expvspred_5cv).to_excel('DT_5fcv_pred.xlsx')

r2_5cv: 0.6216322110769961 rmse_5CV 14.588325802630742 mae_5CV 9.23754452662722


In [6]:
regressor = rfc.fit(Xtrain, Ytrain)
test_predictions = regressor.predict(Xtest)
test_mae = MAE(Ytest,test_predictions)
score_test = regressor.score(Xtest,Ytest)
rmse = np.sqrt(mean_squared_error(Ytest,test_predictions))
print("r2_test:",score_test,"rmse_test",rmse,"mae_test",test_mae)
expvspred_test = {'Exp':Ytest,'Pred':test_predictions}
pd.DataFrame(expvspred_test).to_excel('DT_test_pred.xlsx')

r2_test: 0.7146215862845264 rmse_test 12.870358588942226 mae_test 8.285254330708662
